<a href="https://colab.research.google.com/github/Maxymos-droid/monaco-editor-host/blob/main/processar_pagamentos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import re
import pandas as pd
from PyPDF2 import PdfReader
from google.colab import files

def extract_data_from_pdf(pdf_path, output_excel_path):
    reader = PdfReader(pdf_path)
    dados_gerais = []

    for page_num, page in enumerate(reader.pages):
        text = page.extract_text()

        # Comprovante de Transferência
        if "Comprovante de Transferência" in text:
            tipo = "Transferência"
            nome = re.search(r"nome do recebedor:\s*(.+)", text)
            valor = re.search(r"valor:\s*R\$ ([\d.,]+)", text)
            data_transferencia = re.search(r"data da transferência:\s*(\d{2}/\d{2}/\d{4})", text)
            identificacao = re.search(r"identificação no comprovante:\s*(.*)", text)
            identificacao = identificacao.group(1).strip() if identificacao else ""
            mensagem_match = re.search(r"mensagem ao recebedor:(.*?)(?:\n\S+:|$)", text, re.DOTALL)
            mensagem = mensagem_match.group(1).strip() if mensagem_match else ""
            mensagem = re.split(r"identificação no comprovante", mensagem, maxsplit=1)[0].strip()
            cpf_cnpj_recebedor_match = re.search(r"CPF / CNPJ do recebedor:\s*([\d./-]+)", text)
            cpf_cnpj_recebedor = cpf_cnpj_recebedor_match.group(1).strip() if cpf_cnpj_recebedor_match else ""
            dados_gerais.append([
                page_num + 1,
                data_transferencia.group(1) if data_transferencia else "",
                tipo,
                cpf_cnpj_recebedor,
                nome.group(1).strip() if nome else "",
                "",
                valor.group(1).replace(".", "").replace(",", ".") if valor else "",
                "",
                "",
                "",
                identificacao,
                mensagem
            ])

        # Comprovante de Boleto
        elif "Comprovante de pagamento de boleto" in text:
            tipo = "Pagamento de boleto"
            razao_social_match = re.search(r"Razão Social:\s*(.+?)\s*(\d{2}\.\d{3}\.\d{3}/\d{4}-\d{2})", text)
            if razao_social_match:
                razao_social = razao_social_match.group(1).strip()
                cpf_cnpj = razao_social_match.group(2).strip()
            else:
                razao_social, cpf_cnpj = "", ""
            operacao_efetuada_match = re.search(r"Operação efetuada em\s*(\d{2}/\d{2}/\d{4})", text)
            data_pagamento = operacao_efetuada_match.group(1).strip() if operacao_efetuada_match else ""
            valor_pagamento_match = re.search(r"Valor do boleto \(R\$\);\s*([\d.,]+)", text)
            valor_pago = valor_pagamento_match.group(1).strip().replace(".", "").replace(",", ".") if valor_pagamento_match else "0"
            desconto_match = re.search(r"\(-\)\s*Desconto \(R\$\):\s*([\d.,]+)", text)
            desconto = desconto_match.group(1).strip().replace(".", "").replace(",", ".") if desconto_match else "0"
            juros_match = re.search(r"\(\+\)\s*Juros/Mora/Multa \(R\$\):\s*([\d.,]+)", text)
            if not juros_match:
                juros_match = re.search(r"\(\+\)\s*Mora/Multa \(R\$\):\s*([\d.,]+)", text)
            juros = juros_match.group(1).strip().replace(".", "").replace(",", ".") if juros_match else "0"
            dados_gerais.append([
                page_num + 1,
                data_pagamento,
                tipo,
                cpf_cnpj,
                razao_social,
                "",
                valor_pago,
                juros,
                "",
                "",
                "",
                desconto
            ])

    # Criar DataFrame e salvar no arquivo Excel
    dados_df = pd.DataFrame(dados_gerais, columns=[
        "Nº da Pag do PDF", "Data de Pagamento", "Tipo de Pagamento", "CPF/CNPJ", "Beneficiário",
        "Banco", "Valor", "Juros", "Plano de Contas", "Filial", "Observação 1", "Observação 2"
    ])
    dados_df.to_excel(output_excel_path, index=False)

# Upload do PDF
uploaded = files.upload()

for filename in uploaded.keys():
    pdf_path = filename
    output_excel_path = "pagamentos_organizados.xlsx"
    extract_data_from_pdf(pdf_path, output_excel_path)

# Download do Excel gerado
files.download(output_excel_path)


KeyboardInterrupt: 